In [1]:
%%capture
!pip install selenium
!apt-get update
!apt-get install -y chromium-chromedriver

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

from IPython.display import clear_output
import time

from tqdm.auto import tqdm
import pandas as pd

In [3]:
profile = pd.DataFrame(
    {'Name': [], 'Link': [], 'Keyword': [], 'Citation': [], 'h-index': [], 'i10-index': [],
     'Number of paper': [], 'School':[]})

paper= pd.DataFrame(
    {'Title': [], 'Link': [], 'Authors': [],'Description': [], 'Publisher': [], 'Journal': [], 'Citation': [],
     'Publication date': [], 'Volume':[], 'Issue':[], 'Pages':[],'Inventors':[],'Conference':[]})

chrome_options = Options()
# chrome_options.page_load_strategy = 'none'
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.binary_location = '/content/chromedriver.exe'
driver = webdriver.Chrome(options=chrome_options)
# driver_1 = webdriver.Chrome(options=chrome_options)

def get_profile(link):
    cur_row=0
    driver.get(link)
    next_button = driver.find_element(By.XPATH, '//*[@id="gsc_authors_bottom_pag"]/div/button[2]')
    while 1:
        print(driver.current_url)
        profiles = driver.find_elements(By.XPATH, '//h3[@class="gs_ai_name"]/a')
        keyword=driver.find_elements(By.XPATH, '//div[@class="gs_ai_int"]')
        cite=driver.find_elements(By.XPATH, '//div[@class="gs_ai_cby"]')
        school=driver.find_elements(By.XPATH, '//div[@class="gs_ai_aff"]')
        for i in range(len(profiles)):
            profile.loc[cur_row,'Name']=profiles[i].text
            profile.loc[cur_row,'Link']=profiles[i].get_attribute('href')
            profile.loc[cur_row,'Keyword']=keyword[i].text
            profile.loc[cur_row,'Citation']=cite[i].text
            profile.loc[cur_row,'School']=school[i].text
            cur_row=cur_row+1

        next_button = driver.find_element(By.XPATH, '//*[@id="gsc_authors_bottom_pag"]/div/button[2]')
        if next_button.is_enabled()==False:
            break
        else:
            next_button.click()

get_profile('https://scholar.google.com.au/citations?hl=en&view_op=search_authors&mauthors=VNU-University+of+Engineering+and+Technology&btnG=')
# get_profile('https://scholar.google.com.au/citations?view_op=view_org&hl=en&org=13319503632786364699&after_author=IVsgAQXp__8J&a')


https://scholar.google.com.au/citations?hl=en&view_op=search_authors&mauthors=VNU-University+of+Engineering+and+Technology&btnG=
https://scholar.google.com.au/citations?view_op=search_authors&hl=en&mauthors=VNU-University+of+Engineering+and+Technology&after_author=GIdmAff8__8J&astart=10
https://scholar.google.com.au/citations?view_op=search_authors&hl=en&mauthors=VNU-University+of+Engineering+and+Technology&after_author=9DpTAcP9__8J&astart=20
https://scholar.google.com.au/citations?view_op=search_authors&hl=en&mauthors=VNU-University+of+Engineering+and+Technology&after_author=WNLIAG3-__8J&astart=30
https://scholar.google.com.au/citations?view_op=search_authors&hl=en&mauthors=VNU-University+of+Engineering+and+Technology&after_author=0EseAdD-__8J&astart=40
https://scholar.google.com.au/citations?view_op=search_authors&hl=en&mauthors=VNU-University+of+Engineering+and+Technology&after_author=XreSAC____8J&astart=50
https://scholar.google.com.au/citations?view_op=search_authors&hl=en&mauthor

In [4]:
cur_row=0
for i in tqdm(range(len(profile))):
    if i%50==0:
        driver.close()
        driver = webdriver.Chrome(options=chrome_options)
    driver.get(profile.loc[i,'Link'])
    showmore_button=driver.find_element(By.XPATH, '//div[@id="gsc_bpf"]/button')
    while showmore_button.is_enabled():
        showmore_button.click()
    title=driver.find_elements(By.XPATH, '//td[@class="gsc_a_t"]/a')
    profile.loc[i,'Number of paper']=len(title)
    hindex=driver.find_elements(By.XPATH, '//table[@id="gsc_rsb_st"]//tr[2]/td[2]')
    if len(hindex)>0:
        profile.loc[i,'h-index']=hindex[0].get_attribute("innerHTML")
    i10index=driver.find_elements(By.XPATH, '//table[@id="gsc_rsb_st"]//tr[3]/td[2]')
    if len(hindex)>0:
        profile.loc[i,'i10-index']=i10index[0].get_attribute("innerHTML")
    for j in title:
        paper.loc[cur_row,'Title']=j.text
        paper.loc[cur_row,'Link']=j.get_attribute('href')
        cur_row=cur_row+1

  0%|          | 0/155 [00:00<?, ?it/s]

In [5]:
paper.drop_duplicates(inplace=True)
paper=paper.reset_index(drop=True)

In [6]:
for i in tqdm(range(len(paper))):
    if i%50==0:
        driver.close()
        driver = webdriver.Chrome(options=chrome_options)
    driver.get(paper.loc[i,'Link'])
    meta_data=driver.find_elements(By.XPATH, '//div[@class="gsc_oci_value"]')
    field=driver.find_elements(By.XPATH, '//div[@class="gsc_oci_field"]')
    for j in range(len(field)):
        if field[j].text!='Citation':
            paper.loc[i,field[j].text]=meta_data[j].text
        elif field[j].text=='Citation':
            paper.loc[i,'Citation']=meta_data[j].find_element(By.TAG_NAME,'a').text

  0%|          | 0/2119 [00:00<?, ?it/s]

In [7]:
profile.to_excel('profile.xlsx', index=False)
paper.to_excel('paper.xlsx', index=False)

In [ ]:
profile.loc[60,'Link']

'https://scholar.google.com.au/citations?hl=en&user=GaqY-X4AAAAJ'

In [ ]:
driver = webdriver.Chrome(options=chrome_options)

driver.get('https://scholar.google.com.au/citations?view_op=view_citation&hl=en&user=caFAR1IAAAAJ&cstart=20&pagesize=80&citation_for_view=caFAR1IAAAAJ:Se3iqnhoufwC')
a=driver.find_elements(By.PARTIAL_LINK_TEXT, '[PDF]')
print(len(a))

0


In [ ]:
for i in a:
  print(i.get_attribute("innerHTML"))
  # break

<span class="gs_ico"></span><span class="gs_ia_notf"></span><span class="gs_lbl"></span>
<span class="gs_ico"></span><span class="gs_ia_notf"></span><span class="gs_lbl"></span>

<span class="gs_ico"></span><span class="gs_ia_notf"></span><span class="gs_lbl">My profile</span>
<span class="gs_ico"></span><span class="gs_ia_notf"></span><span class="gs_lbl">My library</span>
<span class="gs_ico"></span><span class="gs_ia_notf"></span><span class="gs_lbl">Metrics</span>
<span class="gs_ico"></span><span class="gs_ia_notf"></span><span class="gs_lbl">Alerts</span>
<span class="gs_ico"></span><span class="gs_ia_notf"></span><span class="gs_lbl">Settings</span>
<span class="gs_ico"></span><span class="gs_ia_notf"></span><span class="gs_lbl">Sign in</span>
<span class="gs_ico"></span><span class="gs_ia_notf"></span><span class="gs_lbl"></span>
Sign in

<span class="gs_rimg gs_pp_sm"><img alt="Nang Dinh Nguyen" sizes="43px" src="https://scholar.googleusercontent.com/citations?view_op=small_ph